In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.combine import SMOTEENN
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import shap
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

In [ ]:
df = pd.read_csv("uterine.csv")

In [ ]:
df.head()

,SL,Age,BMI,Menopause,Cycle Length (days),Flow days,Irregularity,Severe Cramps,Heavy Bleeding,Clotting,...,Acne,Sudden Weight gain/loss,Process Food Consumption,Sugar Consumption,Caffeine Consumption,Sleep schedule (Hours),Stress Level,Physical activity,Diagnosis,Unnamed: 28
0,1,36-45,Normal,No,21-27,2-3,Yes,No,Yes,Yes,...,No,No,Rarely,Very Little,Rarely,8-Jul,Sometimes,Moderate,NaN,NaN
1,2,18-25,Normal,No,<21,2-3,Yes,Yes,Yes,Yes,...,Yes,No,2-3 /week,Moderate,1-2 /day,<4,Sometimes,Moderate,NaN,NaN
2,3,36-45,Overweight,No,<21,2-3,Yes,Yes,No,Yes,...,Yes,No,1/week,Moderate,1-2 /day,<4,Very Often,Moderate,"bulky uterus with uterine fluid, nabothian cyst",NaN
3,4,26-35,Obese,No,>32,>7,Yes,Yes,Yes,Yes,...,No,Yes,2-3/week,High,1-2 /day,6-Apr,Sometimes,Rarely,fatty liver with cystitis with left adnexal cyst,NaN
4,5,26-35,Overweight,No,>32,4-5,Yes,No,No,No,...,Yes,No,2-3/week,Moderate,Rarely,<4,Very Often,Moderate,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   SL                         223 non-null    int64 
 1   Age                        223 non-null    object
 2   BMI                        222 non-null    object
 3   Menopause                  223 non-null    object
 4   Cycle Length (days)        209 non-null    object
 5   Flow days                  208 non-null    object
 6   Irregularity               209 non-null    object
 7   Severe Cramps              209 non-null    object
 8   Heavy Bleeding             209 non-null    object
 9   Clotting                   209 non-null    object
 10  Dizziness                  209 non-null    object
 11  Extreme Fatigue            209 non-null    object
 12  Nausea                     209 non-null    object
 13  Extreme light flow         209 non-null    object
 14  Family His

In [ ]:
df.columns

Index(['SL', 'Age', 'BMI', 'Menopause', 'Cycle Length (days)', 'Flow days',
       'Irregularity', 'Severe Cramps', 'Heavy Bleeding', 'Clotting',
       'Dizziness', 'Extreme Fatigue', 'Nausea', 'Extreme light flow',
       'Family History', 'Previous History', 'Hair Loss', 'Mood Swing',
       'Excess Facial / Body Hair', 'Acne', 'Sudden Weight gain/loss',
       'Process Food Consumption', 'Sugar Consumption', 'Caffeine Consumption',
       'Sleep schedule (Hours)', 'Stress Level', 'Physical activity',
       'Diagnosis', 'Unnamed: 28'],
      dtype='object')

In [ ]:
df.drop('Unnamed: 28', axis=1, inplace=True)
df.info() # Check that the column is gone

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   SL                         223 non-null    int64 
 1   Age                        223 non-null    object
 2   BMI                        222 non-null    object
 3   Menopause                  223 non-null    object
 4   Cycle Length (days)        209 non-null    object
 5   Flow days                  208 non-null    object
 6   Irregularity               209 non-null    object
 7   Severe Cramps              209 non-null    object
 8   Heavy Bleeding             209 non-null    object
 9   Clotting                   209 non-null    object
 10  Dizziness                  209 non-null    object
 11  Extreme Fatigue            209 non-null    object
 12  Nausea                     209 non-null    object
 13  Extreme light flow         209 non-null    object
 14  Family His

In [ ]:
df.columns = [
    'sl', 'age', 'bmi', 'menopause', 'cycle_length_days', 'flow_days',
    'irregularity', 'severe_cramps', 'heavy_bleeding', 'clotting',
    'dizziness', 'extreme_fatigue', 'nausea', 'extreme_light_flow',
    'family_history', 'previous_history', 'hair_loss', 'mood_swing',
    'excess_facial_body_hair', 'acne', 'sudden_weight_change',
    'processed_food_consumption', 'sugar_consumption', 'caffeine_consumption',
    'sleep_hours', 'stress_level', 'physical_activity', 'diagnosis'
]

print(df.columns) # Verify the new names

Index(['sl', 'age', 'bmi', 'menopause', 'cycle_length_days', 'flow_days',
       'irregularity', 'severe_cramps', 'heavy_bleeding', 'clotting',
       'dizziness', 'extreme_fatigue', 'nausea', 'extreme_light_flow',
       'family_history', 'previous_history', 'hair_loss', 'mood_swing',
       'excess_facial_body_hair', 'acne', 'sudden_weight_change',
       'processed_food_consumption', 'sugar_consumption',
       'caffeine_consumption', 'sleep_hours', 'stress_level',
       'physical_activity', 'diagnosis'],
      dtype='object')


In [ ]:
df.shape

(223, 28)

In [ ]:
print(df.columns)

Index(['sl', 'age', 'bmi', 'menopause', 'cycle_length_days', 'flow_days',
       'irregularity', 'severe_cramps', 'heavy_bleeding', 'clotting',
       'dizziness', 'extreme_fatigue', 'nausea', 'extreme_light_flow',
       'family_history', 'previous_history', 'hair_loss', 'mood_swing',
       'excess_facial_body_hair', 'acne', 'sudden_weight_change',
       'processed_food_consumption', 'sugar_consumption',
       'caffeine_consumption', 'sleep_hours', 'stress_level',
       'physical_activity', 'diagnosis'],
      dtype='object')


In [ ]:
print("--- Unique Values in 'age' ---")
print(df['age'].value_counts())

print("\n--- Unique Values in 'bmi' ---")
print(df['bmi'].value_counts())

print("\n--- Unique Values in 'cycle_length_days' ---")
print(df['cycle_length_days'].value_counts())

print("\n--- Unique Values in 'flow_days' ---")
print(df['flow_days'].value_counts())

print("\n--- Unique Values in 'sleep_hours' ---")
print(df['sleep_hours'].value_counts())

--- Unique Values in 'age' ---
age
26-35    75
36-45    60
18-25    60
>45      18
<18       7
45        1
26-32     1
>18       1
Name: count, dtype: int64

--- Unique Values in 'bmi' ---
bmi
Normal         112
Underweight     60
Overweight      40
Obese           10
Name: count, dtype: int64

--- Unique Values in 'cycle_length_days' ---
cycle_length_days
>32      75
28-32    53
<21      45
21-27    27
<21       6
>21       1
<32       1
18-32     1
Name: count, dtype: int64

--- Unique Values in 'flow_days' ---
flow_days
>7     75
4-5    49
6-7    47
2-3    36
>5      1
Name: count, dtype: int64

--- Unique Values in 'sleep_hours' ---
sleep_hours
6-Apr    78
8-Jul    67
<4       49
>8       21
<4        2
6-May     1
7-Jun     1
Name: count, dtype: int64


In [ ]:
# Create a mapping dictionary for ranges and special values
age_map = {
    '26-35': 30.5,
    '36-45': 40.5,
    '18-25': 21.5,
    '>45': 50.0, # Assuming 50 as a reasonable upper bound for analysis
    '<18': 17.0, # Assuming 17 as a reasonable lower bound
    '45': 45.0,
    '26-32': 29.0, # Midpoint of 26-32
    '>18': 19.0  # Assuming 19 as the next integer after >18
}

# Apply the mapping
df['age'] = df['age'].replace(age_map)

# Convert the column to float (This is the final goal)
df['age'] = df['age'].astype(float)

print(df['age'].value_counts())

age
30.5    75
40.5    60
21.5    60
50.0    18
17.0     7
45.0     1
29.0     1
19.0     1
Name: count, dtype: int64


In [ ]:
# 1. Clean the column by removing all leading/trailing whitespace
df['cycle_length_days'] = df['cycle_length_days'].str.strip()

# 2. Define the mapping (same as before)
cycle_map = {
    '>32': 35.0,
    '28-32': 30.0,
    '<21': 20.0,
    '21-27': 24.0,
    '<32': 31.0,
    '>21': 22.0,
    '18-32': 25.0,
}

# 3. Apply the mapping and convert to float
# Note: .replace() handles Null values gracefully, but the string operations above do not.
# We'll apply the .str.strip() to only non-null values to avoid errors if there are nulls.
df['cycle_length_days'] = df['cycle_length_days'].replace(cycle_map).astype(float)


print("\nCleaned cycle_length_days:")
# Show the counts, including the NaN (missing) values, to verify
print(df['cycle_length_days'].value_counts(dropna=False))


Cleaned cycle_length_days:
cycle_length_days
35.0    75
30.0    53
20.0    51
24.0    27
NaN     14
22.0     1
31.0     1
25.0     1
Name: count, dtype: int64


In [ ]:
# 1. Clean the column by removing all leading/trailing whitespace
df['flow_days'] = df['flow_days'].str.strip()

# 2. Define the mapping
flow_map = {
    '>7': 8.0,
    '4-5': 4.5,
    '6-7': 6.5,
    '2-3': 2.5,
    '>5': 6.0,
}

# 3. Apply the mapping and convert to float
df['flow_days'] = df['flow_days'].replace(flow_map).astype(float)

print("\nCleaned flow_days:")
# Show the counts, including the NaN (missing) values, to verify
print(df['flow_days'].value_counts(dropna=False))


Cleaned flow_days:
flow_days
8.0    75
4.5    49
6.5    47
2.5    36
NaN    15
6.0     1
Name: count, dtype: int64


In [ ]:
# 1. Clean the column by removing all leading/trailing whitespace
df['sleep_hours'] = df['sleep_hours'].str.strip()

# 2. Define the mapping
sleep_map = {
    '6-Apr': 5.0,
    '8-Jul': 7.5,
    '<4': 3.0,
    '>8': 9.0,
    '6-May': 5.5,
    '7-Jun': 6.5,
}

# 3. Apply the mapping and convert to float
df['sleep_hours'] = df['sleep_hours'].replace(sleep_map).astype(float)

print("\nCleaned sleep_hours:")
# Show the counts, including the NaN (missing) values, to verify
print(df['sleep_hours'].value_counts(dropna=False))


Cleaned sleep_hours:
sleep_hours
5.0    78
7.5    67
3.0    51
9.0    21
NaN     4
5.5     1
6.5     1
Name: count, dtype: int64


In [ ]:
# Calculate the total number of missing values per column
missing_counts = df.isnull().sum()

# Calculate the percentage of missing values
missing_percent = (missing_counts / len(df)) * 100

# Combine into a summary DataFrame and filter for columns with missing data
missing_summary = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Percent': missing_percent
})

# Display only the columns with missing values, sorted by percentage
print("--- Missing Value Summary (Columns with NaN) ---")
print(missing_summary[missing_summary['Missing Count'] > 0].sort_values(by='Missing Percent', ascending=False))

--- Missing Value Summary (Columns with NaN) ---
                            Missing Count  Missing Percent
family_history                         27        12.107623
flow_days                              15         6.726457
heavy_bleeding                         14         6.278027
irregularity                           14         6.278027
cycle_length_days                      14         6.278027
dizziness                              14         6.278027
clotting                               14         6.278027
extreme_fatigue                        14         6.278027
severe_cramps                          14         6.278027
extreme_light_flow                     14         6.278027
nausea                                 14         6.278027
diagnosis                               7         3.139013
physical_activity                       5         2.242152
sleep_hours                             4         1.793722
caffeine_consumption                    4         1.793722
sugar_c

In [ ]:
# Calculate the median for the numeric columns
median_flow = df['flow_days'].median()
median_cycle = df['cycle_length_days'].median()
median_sleep = df['sleep_hours'].median()

# Fill the missing values with their respective medians
df['flow_days'].fillna(median_flow, inplace=True)
df['cycle_length_days'].fillna(median_cycle, inplace=True)
df['sleep_hours'].fillna(median_sleep, inplace=True)

# Verify the imputation
print("Flow Days NaNs after imputation:", df['flow_days'].isnull().sum())
print("Cycle Length NaNs after imputation:", df['cycle_length_days'].isnull().sum())
print("Sleep Hours NaNs after imputation:", df['sleep_hours'].isnull().sum())

Flow Days NaNs after imputation: 0
Cycle Length NaNs after imputation: 0
Sleep Hours NaNs after imputation: 0


In [ ]:
# Identify all categorical columns with missing values (excluding the ones we just did)
cat_cols_with_nan = missing_summary.index[missing_summary['Missing Count'] > 0].tolist()

# Remove the numeric columns we already handled to avoid errors
for col in ['flow_days', 'cycle_length_days', 'sleep_hours']:
    if col in cat_cols_with_nan:
        cat_cols_with_nan.remove(col)

print("\nImputing these categorical columns with the MODE:")
print(cat_cols_with_nan)

# Loop through and impute each column with its mode
for col in cat_cols_with_nan:
    # Calculate the mode (most frequent value). [0] selects the first mode if there are ties.
    mode_value = df[col].mode()[0]

    # Fill the missing values
    df[col].fillna(mode_value, inplace=True)

# Final verification check: All columns should now have 0 missing values
print("\nFinal Check - Total Remaining NaNs:", df.isnull().sum().sum())

# Display the information of the entire DataFrame
df.info()


Imputing these categorical columns with the MODE:
['bmi', 'irregularity', 'severe_cramps', 'heavy_bleeding', 'clotting', 'dizziness', 'extreme_fatigue', 'nausea', 'extreme_light_flow', 'family_history', 'previous_history', 'hair_loss', 'mood_swing', 'excess_facial_body_hair', 'acne', 'sudden_weight_change', 'processed_food_consumption', 'sugar_consumption', 'caffeine_consumption', 'stress_level', 'physical_activity', 'diagnosis']

Final Check - Total Remaining NaNs: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sl                          223 non-null    int64  
 1   age                         223 non-null    float64
 2   bmi                         223 non-null    object 
 3   menopause                   223 non-null    object 
 4   cycle_length_days           223 non-null    float64
 5   flow_days        

In [ ]:
# Create a list of columns that are known to be binary (Yes/No, etc.)
binary_cols = [
    'menopause', 'irregularity', 'severe_cramps', 'heavy_bleeding', 'clotting',
    'dizziness', 'extreme_fatigue', 'nausea', 'extreme_light_flow',
    'family_history', 'previous_history', 'hair_loss', 'mood_swing',
    'excess_facial_body_hair', 'acne', 'sudden_weight_change'
]

# Define the mapping: 1 for 'Yes', 0 for 'No'
binary_map = {'Yes': 1, 'No': 0}

# Apply the mapping to all binary columns
for col in binary_cols:
    # First, convert to uppercase to handle potential case inconsistencies (e.g., 'yes' vs 'Yes')
    df[col] = df[col].str.title().replace(binary_map)

# Special case: The 'sl' column is a sequence number and should be dropped or left as is.
# We'll drop it as it has no predictive power.
df.drop('sl', axis=1, inplace=True)

print("Check a binary column after encoding (Irregularity):")
print(df['irregularity'].value_counts())

Check a binary column after encoding (Irregularity):
irregularity
1    161
0     62
Name: count, dtype: int64


In [ ]:
# Identify the remaining object columns (excluding the ones just encoded)
# The remaining object columns should be:
# bmi, processed_food_consumption, sugar_consumption, caffeine_consumption,
# stress_level, physical_activity, diagnosis

cols_to_onehot = df.select_dtypes(include='object').columns.tolist()

print("\nColumns to be One-Hot Encoded:", cols_to_onehot)

# Apply One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=cols_to_onehot, drop_first=True) # drop_first=True avoids multicollinearity

print("\nShape after encoding:", df_encoded.shape)
print("New columns after encoding (First 50):", df_encoded.columns[:50].tolist())


Columns to be One-Hot Encoded: ['bmi', 'severe_cramps', 'family_history', 'mood_swing', 'processed_food_consumption', 'sugar_consumption', 'caffeine_consumption', 'stress_level', 'physical_activity', 'diagnosis']

Shape after encoding: (223, 193)
New columns after encoding (First 50): ['age', 'menopause', 'cycle_length_days', 'flow_days', 'irregularity', 'heavy_bleeding', 'clotting', 'dizziness', 'extreme_fatigue', 'nausea', 'extreme_light_flow', 'previous_history', 'hair_loss', 'excess_facial_body_hair', 'acne', 'sudden_weight_change', 'sleep_hours', 'bmi_Obese', 'bmi_Overweight', 'bmi_Underweight', 'severe_cramps_1', 'severe_cramps_ Yes', 'family_history_1', 'family_history_Yes ', 'mood_swing_1', 'mood_swing_Np', 'processed_food_consumption_1/weeek', 'processed_food_consumption_1/week', 'processed_food_consumption_2-3 /week', 'processed_food_consumption_2-3/week', 'processed_food_consumption_Daily', 'processed_food_consumption_Rarely', 'sugar_consumption_High', 'sugar_consumption_Mo

In [ ]:
# --- Fixing Binary Columns (Addressing extra spaces or inconsistent capitalization) ---

# 1. Standardize case and remove whitespace from all relevant categorical columns
cols_to_strip = df.select_dtypes(include='object').columns.tolist()
for col in cols_to_strip:
    # Convert to title case and strip whitespace to standardize
    df[col] = df[col].astype(str).str.title().str.strip()

# 2. Manual Fixes for identified typos/inconsistencies

# Fix Processed Food Consumption
df['processed_food_consumption'].replace(
    {'2-3 /Week': '2-3/Week', '1/Weeek': '1/Week'},
    inplace=True
)

# Fix Caffeine Consumption
df['caffeine_consumption'].replace(
    {'1-2/Ay': '1-2/Day', '1-2/Day0': '1-2/Day'},
    inplace=True
)

# Fix BMI categories (ensure standard capitalization)
df['bmi'] = df['bmi'].str.title()

In [ ]:
# Re-apply Binary Encoding (for columns that should only have 1/0)
# We will check if the previous binary columns still have 'object' type before proceeding.
# Let's verify by just printing the object columns again:
cols_to_onehot_new = df.select_dtypes(include='object').columns.tolist()

print("\n--- Columns to be One-Hot Encoded (REVISED) ---")
print(cols_to_onehot_new)

# Re-apply One-Hot Encoding to the clean object columns
df_encoded = pd.get_dummies(df, columns=cols_to_onehot_new, drop_first=True)

print("\nShape after FINAL encoding:", df_encoded.shape)
print("New column count (should be lower):", len(df_encoded.columns))


--- Columns to be One-Hot Encoded (REVISED) ---
['bmi', 'severe_cramps', 'family_history', 'mood_swing', 'processed_food_consumption', 'sugar_consumption', 'caffeine_consumption', 'stress_level', 'physical_activity', 'diagnosis']

Shape after FINAL encoding: (223, 186)
New column count (should be lower): 186
